## Task 1 - Third-Order Letter Approximation Model

The below code takes in all of the txt files in texts/, filtering out special & numerical characters, then counting how often each sequence of three characters appears and compiles it into a dictionary.

<hr>

In [66]:
# Imports and Global Variables
import os
import re
import random
import json

sanitizedTexts = []
trigrams = {}

### sanitizeText
<hr>
This function takes in a text file (ideally from Project Gutenberg), and cleans it up for use in a later function.
This involves removing the pre and post amble, removing special characters, and multi-blank lines.
The idea of this is to create text perfect for turning into trigrams - no niche special characters,
no multi-blank lines flooding the trigrams with {"\n\n\n"}, and no legalese from the Project Gutenberg postamble.

Here are some of the more complex lines explained in more depth:

This line takes the text and removes the Project Gutenberg pre and post amble. To accomplish this, 
the preamble and postamble variables are defined with the text at the end of the preamble, and the beginning of the postamble.
Python's method of slicing (the : character between two indexes) **[3]** requires us to first find where in the string these characters are, which is why .index() is used. +len() is used to get to the end of the preamble, so that it isn't included.

An example of how index and slicing works is below. .index() gets the index (that is, how far into the string the substring is) and slicing gets the substring between two indexes.

In [ ]:
print('abcabcabc'.index('b')) # Result = 1.
print('abc123abc'[3:6]) # Result = 123


The result of the combination is an extraction of all of the text in between the preamble and postamble:


```python
    sanitizedText = (text[text.index(preamble)+len(preamble):text.index(postamble)]) # [3.1]
```
<hr>

Now, we can move on to removing special characters and multi-blank lines. We can do this through Python Regex **[4]**, enabling us to set a "filter" for only the text we want to keep. This is what's being done in the first line:
- re.sub means that we're replacing things that don't fit into our filter with the second string, empty in this case.
- The [] designates a regex set, allowing us to define ranges of characters.
- The ^ character is a complement - it means that we're substituing any characters that *don't* fit into our set.
- a-zA-Z refers to all alphabetical lowercase & uppercase characters between A and Z.
- \\s refers to spaces, . refers to periods, and \n refers to new lines. These are the only sepcial characters we want to keep.

So after running through this first line, any commas, colons or the like are removed from the string.

```python
    sanitizedText = re.sub("[^a-zA-Z\\s.\n]", "", sanitizedText) # [4.1]
```

The second line is more of the same - this time, it's looking for multiple instances of blank lines in a row. A lot of the Project Gutenberg books feature this, and if we allowed them into the Trigrams, we'd end up with a higher than average liklihood of the approximation generating 3 or 4 new lines after each line break.

_r"\n\s*\n"_ is a pattern looking for two new lines with nothing but spaces/new lines in between them - for example, if there were an instance of 3 new lines, one with a space in the middle, this would remove it and replace it with just one new line.

We're subbing these examples with just one new line - we don't want our approximation to be all one blob of a line, so having some new lines is useful for making it look more like language.

```python
    sanitizedText = re.sub(r"\n\s*\n", "\n", sanitizedText) # [4.2]
```
<hr>

Finally, our text is almost ready to be returned. We just have to perform some simple String Operations **[5]**, which Python handily provides. 
- .upper() converts all our text to uppercase, making the Trigrams more consistent later. 
- .strip() removes any leftover whitespace at the beginning/end of each text.

```python
    return sanitizedText.upper().strip()
```

In [68]:
# Takes in a Project Gutenberg text and cleans it up for use in produceTrigrams
# Removes preamble and postamble,
def sanitizeText(text):
    preamble = " ***" # Ending of the preamble
    postamble = "*** END OF " # Beginning of the postamble.

    # [3] Strips out the preamble and postamble from a text by
    # creating a string in between the preamble and postamble variables.
    sanitizedText = (text[text.index(preamble)+len(preamble):text.index(postamble)]) 
    sanitizedText = re.sub("[^a-zA-Z\\s.\n]", "", sanitizedText) # [4.1]
    sanitizedText = re.sub(r"\n\s*\n", "\n", sanitizedText) # [4.2]

    return sanitizedText.upper().strip() # Set to uppercase, and remove trailing/leading spaces. [5]

### produceTrigrams
<hr>
This function takes in a list of our newly-sanitized texts and creates a dict of Trigrams, which end up looking something like:
{"ABC": 123, "DEF": 342, ...} This tells us how often a set of three characters occurs in our text, which we can use later on to build our approximation.

First we iterate over the texts with a for loop, then we *enumerate* over them **[6]**. This means we have a handy *counter* alongside our character while we go through each text. We'll need this for string indexes in a bit.

```python
    for text in texts:
        for counter, c in enumerate(text)
```


Now we're going through every character in every text. We need to pull out three characters at a time. For example, "THIS IS THE END" would become "THI", "HIS", "IS " and so on. This can be done easily enough with slicing using our counter:

```python 
    currentText = text[counter:counter+3]
```

This gives us three characters - the current character we're on, and the two following ones. This is the current effectively the current trigram. Now we just need to check something before adding it to our dictionary:
If the trigram already exists in our dictionary, we just increment it's value. If we have "ABC": 1 and run into "ABC" again, we just get "ABC": 2. We should also be sure to check that the Trigram is actually three characters long - when reaching the end of the text, we may end up grabbing only one or two characters. We can cut these out here as we go.
```python
    if(len(currentText) < 3):
        print("Found shorter trigram, "+currentText+", disregarding.")
    if(trigrams.get(currentText) != None): # If the trigram key already exists, increment it's value.
        trigrams[currentText] = trigrams[currentText] + 1
```

If it doesn't yet exist, though, we'll need to initialize it. This is done like so - we just set it's value to 1.
```python
    else: # If the trigram hasn't been added to the dict yet, add it with the value of 1.
        trigrams[currentText] = 1
```

After iterating through every character in every text, we now have a full dictionary of trigrams to return. 

In [69]:
# Takes in a set of sanitized texts and turns them into a dictionary of trigrams, in the format {"ABC": 123}.
def produceTrigrams(texts):
    trigrams = {}

    for text in texts:
        for counter, c in enumerate(text): #Enumerate over the text, used to keep a counter of what character we're on. [6]
            # text[counter:counter+3] gets 3 characters - the current one, and the two afterwards. [3]
            # This basically gives us the "current" trigram.
            currentText = text[counter:counter+3]
            if(len(currentText) < 3):
                print("Found shorter trigram, "+currentText+", disregarding.")
            elif(trigrams.get(currentText) != None): # If the trigram key already exists, increment it's value.
                trigrams[currentText] = trigrams[currentText] + 1
            else: # If the trigram hasn't been added to the dict yet, add it with the value of 1.
                trigrams[currentText] = 1

    return trigrams
                

### Getting our Trigrams Dict
<hr>
Now we have two functions for creating a dictionary of trigrams, we need to pull out some text from our files to use them.

I used os.scandir() to achieve this, part of the OS library **[2]**. This let's us iterate over every file it finds in a certain folder (texts in this case).

```python
    for title in os.scandir("texts"):
```

I'd like to be able to pull in everything from the /texts/ directory, but first I need to make sure they're all text files. To do this, I use another string operation - find(). This looks for the string ".txt", and only opens up a file if it's present.

```python
    if((title.name.find(".txt")) != -1):
```

Now that we're sure the file is a text file, we can open() it **[1]**, enabling us to call read() on it, which pulls out the text content of the file. Then, we .close() the file for safety.
Text is sanitized by the sanitizeText method as soon as it's read from .read(), and then added to a list for use later.

```python
    fileContent = open(title) # [1]
    sanitizedTexts.append(sanitizeText(fileContent.read())) # Sanitize it & add it to the sanitizedTexts list.
    fileContent.close()
```

Finally, once all the text files have been read, we can pass our newly created sanitizedTexts list into produceTrigrams, and get our dictionary.
```python
    trigrams = produceTrigrams(sanitizedTexts)
```

In [ ]:
for title in os.scandir("texts"): # [2]
    if((title.name.find(".txt")) != -1): # Only attempt to open + add the text file if it's a .txt
        fileContent = open(title) # [1]
        sanitizedTexts.append(sanitizeText(fileContent.read())) # Sanitize it & add it to the sanitizedTexts list.
        fileContent.close()

trigrams = produceTrigrams(sanitizedTexts)
print(trigrams)

## Task 2 - Third-Order Letter Approximation Generation

The below code takes the list of trigrams and uses it to generate an approximation of english, deciding randomly weighted by the frequency of the next character, looking back at the previous two.
<hr>

First, we declare a current_string of "TH". These are the first two characters the approximation will use, as most texts start with these two characters in one way or another.

In [71]:
current_string = "TH"

This is the loop where we generate the approximation, and it's doing a lot. It executes 10000 times to generate a text 10000 characters (incl whitespaces) long.

The first line in the loop is a list comprehension, adapted from one meant to generate second-order approximations.
List Comprehensions have an input, range and output and are used to easily iterate through lists, dictionaries and sets and perform some sort of operation on them.

What the below one is doing is taking the trigrams (for x in trigrams.keys()) and checking for which ones have the same first two letters as the last two letters of the current_string (if x[0:2 == current_string[i:]]). The last character of the trigram is taken (which would be the potential next character of current_string), and then zipped up with its associated weight into the tris and weights variables.

This gives us a list of the potential next characters as well as the weights for each of them, which we can use for the next line. This is done every loop - every character, we check what the last characters we generated were, and generate a list of potential next characters and weights.

```python
    tris, weights = list(zip(*[(x[2], trigrams[x]) for x in trigrams.keys() if x[0:2] == current_string[i:]]))
```

<hr>

Now we need to actually add the next character to the current string. This is done using random.choices, which takes in the list of potential next characters, and their respective weights.

The below line may look as bulky as the last one, but it's actually much simpler. ''.join simply means that we're appending whatever character we generate, not replacing anything. We need to pass this two strings: one of them being the current one, and the other being our randomly generated character. 

This is where random.choices comes in. We pass it the tris, weights, and a population (how many things to generate) of 1. Because this generates as a list, we also need to give it a [0] to tell it that we only want the first value from it (which will also be the only one, as it has a population of 1).

```python
    current_string = ''.join([current_string, (random.choices(list(tris), weights=weights, k=1)[0])])
```

Once this has run (it may take about 30ish seconds due to some inefficiency), current_string will contain a 10000 character long approximation of the english language.

In [ ]:
for i in range(10001):
    tris, weights = list(zip(*[(x[2], trigrams[x]) for x in trigrams.keys() if x[0:2] == current_string[i:]])) # [7]
    current_string = ''.join([current_string, (random.choices(list(tris), weights=weights, k=1)[0])]) # [7]

print(current_string)

## Task 3 - Analyze The Model

Now we need to see how effective our model is. To do this, we figure out what % of the words generated are real words by comparing it to a list.
<hr>

First, we open up the words.txt file provided. We just need to read this in, and split it up into a list we can compare our approximation to. We can close the file when we're done.

In [73]:
words_file = open("words.txt", "r")
words_list = words_file.read().split("\n") # Read the file and split it up by line into a list.
words_file.close() 

Next, we create approx_words by splitting up our approximation (stored in current_string).


Then we can move on to our loop to check the accuracy of our approximation, which is actually relatively simple. We create a correct_words and set it at 0, which we'll use to measure how well the approximation holds up. Then, we just loop over every word in approx_words and check if it's in words_list. If it is, we can increment correct_words by 1. This gives us the number of words that are found in both - the number of words that are real english.

In [74]:
approx_words = current_string.split() # Handily, .split() by default handles all forms of whitespace. [5]

correct_words = 0

for word in approx_words: # Check every word in the approximation against the words list, and increment correct_words if correct.
    if word in words_list:
        correct_words = correct_words + 1

Now we can measure the percentage of real words in our approximation by doing correct_words/len(approx_words)*100. I rounded this to two decimal places for readability. From running it a few times, this usually ranges from about 35-38% english words.

In [ ]:
percentage = round(((correct_words/len(approx_words))*100), 2)
print(percentage, "% real words.")

## Task 4 - Export Model
Finally, we export the model using python's json library.
<hr>

This final part is very simple - all we do is open the trigrams.json file, write to it, and close it again.

I used the json.dumps() function to output to json in a readable format, which takes in the trigrams model. It's also sorted and indented to organise it a bit.

In [78]:
jsonFile = open("trigrams.json", "w")
jsonFile.write(json.dumps(trigrams, sort_keys=True, indent=3)) #[8]
jsonFile.close

<hr>

## References

- [1] - Reading text files in Python: https://www.w3schools.com/python/python_file_open.
- [2] - Using Python's OS Module: https://www.geeksforgeeks.org/how-to-iterate-over-files-in-directory-using-python/
- [3] - Slicing text in Python: https://python-reference.readthedocs.io/en/latest/docs/brackets/slicing.html
    - [3.1] - Finding a string between two substrings (used for removing Pre/Postamble): https://stackoverflow.com/a/51456576
- [4] - Python Regex Library Docs: https://docs.python.org/3/library/re.html
    - [4.1] - Using regex in Python: https://www.w3schools.com/python/python_regex.asp
    - [4.2] - Removing multi-blank lines in a text: https://stackoverflow.com/a/28902081
- [5] - Python String Methods: https://docs.python.org/3.4/library/stdtypes.html#string-methods
- [6] - Python enumerator/counter: https://docs.python.org/3/library/functions.html#enumerate
- [7] - Second Order Approximation Method from Notes: https://github.com/ianmcloughlin/2425_emerging_technologies/blob/main/02_language_models.ipynb
- [8] - Python JSON Library Documentation: https://docs.python.org/3/library/json.html

<hr>